# Mapping BNF codes to dm+d - Update November 2024

We have had a request from NHS England:

>Our initial need is to have a reference file that can be used to map data in BNF code form (from NHS BSA) to drug information in dm+d (SNOMED) form (at VMP/AMP level but also with VTM information).

We hold this information in the BQ database, and should be able to create a query to deliver this need.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib.dates import  DateFormatter
%matplotlib inline
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps
import datetime
import openpyxl

## Create data from BigQuery

In [ ]:
sql = """
  SELECT
  "vmp" AS type, # create type column, shows whether VMP or AMP
  vmp.id AS id, # VMP code
  vmp.nm AS nm, # VMP name
  vmp.vtm AS vtm, # VTM code
  vtm.nm AS vtm_nm, # VTM name
  bnf_code, # BNF code
  vpidprev AS vmp_previous, # Previous VMP code
  vpiddt AS vmp_previous_date # Date that previous VMP code changed
FROM
  ebmdatalab.dmd.vmp_full AS vmp
LEFT OUTER JOIN
  dmd.vtm AS vtm
ON
  vmp.vtm = vtm.id

UNION ALL # join VMP and AMP tables together to form single table
SELECT
  "amp" AS type,
  amp.id,
  amp.descr,
  vmp.vtm AS vtm,
  vtm.nm AS vtm_nm,
  amp.bnf_code AS bnf_code,
  NULL AS amp_previous,
  NULL AS amp_previous_date
FROM
  ebmdatalab.dmd.amp_full AS amp
INNER JOIN
  dmd.vmp AS vmp # join VMP to AMP table to get VMP codes to obtain VTM information
ON
  amp.vmp = vmp.id
LEFT OUTER JOIN
  dmd.vtm AS vtm
ON
  vmp.vtm = vtm.id
  """

exportfile = os.path.join("..","data","dmd_df.csv")
dmd_df = bq.cached_read(sql, csv_path=exportfile, use_cache=False)
exportfile2 = os.path.join("..","data","bnf_to_dmd.csv")
dmd_df['id'] = dmd_df['id'].astype('Int64')  # ensure csv is integer
dmd_df['vtm'] = dmd_df['vtm'].astype('Int64') # ensure csv is integer
dmd_df['vmp_previous'] = dmd_df['vmp_previous'].astype('Int64') # ensure csv is integer
exportfile2 = os.path.join("..","data","bnf_to_dmd.csv")
dmd_df.to_csv(exportfile2) # export integer version

Downloading: 100%|██████████| 182104/182104 [00:19<00:00, 9151.58rows/s]


In [5]:
dmd_df.head()

,type,id,nm,vtm,vtm_nm,bnf_code,vmp_previous,vmp_previous_date
0,vmp,68461003,Lubricant gels,NaN,None,None,3.48531e+15,2006-01-04
1,vmp,134619001,Doxorubicin (liposomal) 50mg/25ml solution for...,NaN,None,None,NaN,NaT
2,vmp,134754002,Carmellose 1% eye drops 0.4ml unit dose,775072005.0,Carmellose,None,NaN,NaT
3,vmp,318076002,Spironolactone 5mg/5ml oral suspension sugar free,777603002.0,Spironolactone,None,NaN,NaT
4,vmp,318077006,Spironolactone 10mg/5ml oral suspension sugar ...,777603002.0,Spironolactone,None,NaN,NaT


As we can see from above we appear to have successfully imported all `VMPs` and `AMPs`.  However, there are some products which either do not have a `VTM` or `bnf_code`.  We will explore this further below.

#### Check data with 12 months of primary care prescribing data

Importing prescribing data from BigQuery to check the impact of "missing" data

In [6]:
sql = """
SELECT
  bnf_code,
  bnf_name,
  SUM(items) AS items
FROM
  ebmdatalab.hscic.normalised_prescribing AS rx
WHERE
  month BETWEEN '2023-09-01'
  AND '2024-08-01'
GROUP BY
  bnf_name,
  bnf_code
  """

exportfile = os.path.join("..","data","rx_df.csv")
rx_df = bq.cached_read(sql, csv_path=exportfile, use_cache=True)

In [7]:
rx_df.head()

,bnf_code,bnf_name,items
0,0202020L0AABBBB,Furosemide 20mg tablets,4268985
1,0106040G0AAAAAA,Lactulose 3.1-3.7g/5ml oral solution,2455918
2,040201060AAACAC,Olanzapine 10mg tablets,738192
3,0601011A0BDABAC,Trurapi 100units/ml inj 3ml pre-filled Solosta...,93493
4,1502010J0AABDBD,Lidocaine 50mg/5ml (1%) solution for injection...,97540


We can now merge the two dataframes on BNF code

In [8]:
test_df = pd.merge(dmd_df, rx_df, left_on='bnf_code', right_on='bnf_code', how='right')

In [9]:
test_df.head()

,type,id,nm,vtm,vtm_nm,bnf_code,vmp_previous,vmp_previous_date,bnf_name,items
0,vmp,7.217100e+13,Tetanus immunoglobulin human 250unit solution ...,777724003.0,Tetanus immunoglobulin human,1405020A0AAABAB,3.33769e+08,2022-01-12,Tetanus immunoglobulin human 250unit inj vials,44
1,amp,4.953011e+15,Tetanus immunoglobulin human 250unit solution ...,777724003.0,Tetanus immunoglobulin human,1405020A0AAABAB,NaN,NaT,Tetanus immunoglobulin human 250unit inj vials,44
2,vmp,1.160810e+14,Imiquimod 3.75% cream 250mg sachets,776315001.0,Imiquimod,1307000V0AAABAB,4.44988e+08,2022-07-29,Imiquimod 3.75% cream 250mg sachets,985
3,vmp,1.162810e+14,Lidocaine 5% medicated plasters,776528009.0,Lidocaine,1502010J0AAELEL,3.76491e+08,2022-07-29,Lidocaine 5% medicated plasters,103102
4,amp,4.278981e+16,Lidocaine 5% medicated plasters (Grunenthal Ltd),776528009.0,Lidocaine,1502010J0AAELEL,None,NaT,Lidocaine 5% medicated plasters,103102


We can now check by seeing which items prescribed which don't have a corresponding `dm+d` code

In [10]:
test_df_no_dmd = test_df[test_df['id'].isnull()].sort_values(by='items', ascending=False) # filter only prescribing which has a null VMP or AMP
test_df_no_dmd.head()

,type,id,nm,vtm,vtm_nm,bnf_code,vmp_previous,vmp_previous_date,bnf_name,items
105736,NaN,NaN,NaN,NaN,NaN,190201000AABLBL,NaN,NaT,Exception Handler Unspecified Item,221383
105738,NaN,NaN,NaN,NaN,NaN,190201000AABPBP,NaN,NaT,Exception Handler High Volume Unspecified Item,65058
105737,NaN,NaN,NaN,NaN,NaN,190201000AABNBN,NaN,NaT,Exception Handler Discount Not Deducted Item,26390
105743,NaN,NaN,NaN,NaN,NaN,0407020A0AACDCD,NaN,NaT,Fentanyl 400microgram sublingual tablets,16
105742,NaN,NaN,NaN,NaN,NaN,0407020A0AACFCF,NaN,NaT,Fentanyl 67microgram sublingual tablets,3


We can see from the above there are very few items apart from "unspecified item", which by definition cannot have a BNF code.

The other part of the request was to link `VTM` codes.  We can also check which drugs do not link to a `VTM`.

In [11]:
test_vtm_no_dmd  = test_df[test_df['vtm'].isnull()].sort_values(by='items', ascending=False)

In [12]:
test_vtm_no_dmd.head(30)

,type,id,nm,vtm,vtm_nm,bnf_code,vmp_previous,vmp_previous_date,bnf_name,items
101895,amp,3.909461e+16,FreeStyle Libre 2 Sensor (Abbott Laboratories ...,NaN,None,21480000101,None,NaT,FreeStyle Libre 2 Sensor,2968952
62884,amp,1.742061e+16,Laxido Orange oral powder sachets sugar free (...,NaN,None,0106040M0BCACAA,NaN,NaT,Laxido Orange oral powder sachets sugar free,2778960
880,amp,2.239391e+16,Macrogol compound oral powder sachets sugar fr...,NaN,None,0106040M0AAAAAA,None,NaT,Macrogol compound oral powder sachets NPF suga...,2149596
884,amp,2.840711e+16,Macrogol compound oral powder sachets sugar fr...,NaN,None,0106040M0AAAAAA,None,NaT,Macrogol compound oral powder sachets NPF suga...,2149596
881,amp,2.267331e+16,Macrogol compound oral powder sachets sugar fr...,NaN,None,0106040M0AAAAAA,None,NaT,Macrogol compound oral powder sachets NPF suga...,2149596
882,amp,2.437791e+16,Macrogol compound oral powder sachets sugar fr...,NaN,None,0106040M0AAAAAA,None,NaT,Macrogol compound oral powder sachets NPF suga...,2149596
883,amp,2.455401e+16,Macrogol compound oral powder sachets sugar fr...,NaN,None,0106040M0AAAAAA,None,NaT,Macrogol compound oral powder sachets NPF suga...,2149596
879,amp,2.179041e+16,Macrogol compound oral powder sachets sugar fr...,NaN,None,0106040M0AAAAAA,None,NaT,Macrogol compound oral powder sachets NPF suga...,2149596
875,vmp,3.591211e+15,Macrogol compound oral powder sachets NPF suga...,NaN,None,0106040M0AAAAAA,NaN,NaT,Macrogol compound oral powder sachets NPF suga...,2149596
876,amp,1.550391e+16,Macrogol compound oral powder sachets sugar fr...,NaN,None,0106040M0AAAAAA,NaN,NaT,Macrogol compound oral powder sachets NPF suga...,2149596


In [13]:
group_vtm_no_dmd = test_vtm_no_dmd.groupby(['bnf_name'])[['items']].mean().sort_values(by='items', ascending=False)

In [14]:
group_vtm_no_dmd.head(30)

,items
bnf_name,
FreeStyle Libre 2 Sensor,2968952
Laxido Orange oral powder sachets sugar free,2778960
Macrogol compound oral powder sachets NPF sugar free,2149596
Dermol 500 lotion,2049895
Epimax original cream,1416706
Otomize ear spray,1361262
Medi Derma-S barrier cream,914676
Vitamin B compound strong tablets,911934
GlucoRx Nexus testing strips,853344


The largest number of prescribing items with a `NULL` `VTM` are either a) where they are not drugs, but appliances or devices (such as Freestyle Libre), OR where the drug has more than 3 ingredients.  In this case (such as Laxido) no VTM is assigned to the formulation in the dm+d.  Therefore it appears that the table accurately reflects what the dm+d says.